# 🛠️ Preventative Maintenance AI Model (Simulated on macOS)
This notebook simulates an AI model designed for predictive maintenance, originally targeted for STM32 embedded deployment. We reconstruct a version that can run locally on a macOS system using Python and TensorFlow.

It covers:
- Creation of a sample 1D CNN model using Keras
- Generation of mock vibration sensor data
- Training the model to distinguish between 'normal' and 'faulty' conditions
- Saving and converting the model to TensorFlow Lite format
- Running inference using TFLite (via TensorFlow)

Let's get started!


## 🔧 Step 1: Create the AI Model
We define a simple 1D Convolutional Neural Network using Keras. This model mimics the kind of lightweight model that could be deployed on microcontrollers for vibration-based anomaly detection.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define a function to create the model
def create_mock_model(input_shape=(128, 3)):
    # Sequential model: good for simple stack of layers
    model = models.Sequential([
        layers.Input(shape=input_shape),                     # Input layer expects 128 timesteps, 3 features (axes)
        layers.Conv1D(16, 3, activation='relu'),             # First 1D convolutional layer with 16 filters
        layers.MaxPooling1D(2),                              # Downsample by factor of 2
        layers.Conv1D(32, 3, activation='relu'),             # Second convolutional layer
        layers.GlobalMaxPooling1D(),                         # Reduce to 1D by taking max across time
        layers.Dense(64, activation='relu'),                 # Fully connected dense layer
        layers.Dense(1, activation='sigmoid')                # Output layer: sigmoid for binary classification
    ])
    # Compile model with loss and optimizer
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create the model
model = create_mock_model()

# Print summary of the architecture
model.summary()

## 📊 Step 2: Generate Synthetic Sensor Data
We simulate accelerometer data for two classes:
- **Normal** operation
- **Abnormal** condition (indicating possible failure)

Each sample has 128 time steps and 3 axes (X, Y, Z) to match typical 3-axis sensor input.

In [ ]:
import numpy as np

# Create 10 normal samples: low mean and low variance
normal_data = np.random.normal(loc=0.0, scale=1.0, size=(10, 128, 3))

# Create 10 abnormal samples: higher mean and variance to simulate more vibration
abnormal_data = np.random.normal(loc=2.0, scale=1.5, size=(10, 128, 3))

# Combine the data into one dataset
X = np.concatenate([normal_data, abnormal_data])

# Labels: 0 for normal, 1 for abnormal
y = np.array([0]*10 + [1]*10)

## 🧠 Step 3: Train the Model
Now, we train the model on our synthetic dataset. Since it's a small mock dataset, we use a few epochs for quick iteration.

In [ ]:
# Train the model using the synthetic data
model.fit(X, y, epochs=10, batch_size=4, verbose=1)

## 💾 Step 4: Save the Model
We save the model in native Keras format, which can be reused or converted for deployment.

In [ ]:
# Save the trained model to file
model.save("preventative_maintenance_model.keras")

## 🔁 Step 5: Convert to TensorFlow Lite
TensorFlow Lite allows us to run inference on lightweight devices or in optimized environments.

Here, we convert the Keras model to `.tflite` format.

In [ ]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to disk
with open("preventative_maintenance_model.tflite", "wb") as f:
    f.write(tflite_model)

## 🤖 Step 6: Run Inference with TensorFlow Lite
Now, we load the `.tflite` model and run inference on one of the abnormal samples.

> 💡 We're using full TensorFlow here instead of `tflite_runtime` for compatibility on macOS.

In [ ]:
# Import interpreter from full TensorFlow
from tensorflow.lite.python.interpreter import Interpreter

# Load the model
interpreter = Interpreter(model_path="preventative_maintenance_model.tflite")
interpreter.allocate_tensors()

# Get input/output tensor info
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare one abnormal sample for prediction
sample = abnormal_data[0:1].astype(np.float32)

# Run inference
interpreter.set_tensor(input_details[0]['index'], sample)
interpreter.invoke()

# Get output
output = interpreter.get_tensor(output_details[0]['index'])
print("Prediction (1 = needs maintenance):", output)